In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
import pytorch_lightning as pl
from pytorch_lightning.metrics.sklearns import R2Score

import pandas as pd
import numpy as np
import joblib
from pathlib import Path

from sklearn.preprocessing import StandardScaler

# Custom
from project.film_model import LinearBlock, FiLMGenerator

## FiLM Structures

In [3]:
from project.film_model import FiLMNetwork, ConcatNetwork

In [5]:
bar = ConcatNetwork(978, 513)

In [6]:
bar.hparams

"batch_size":    2048
"conds_sz":      513
"inputs_sz":     978
"learning_rate": 0.001
"metric":        R2Score()

In [8]:
bar.inputs_emb.out_sz + bar.conds_emb.out_sz

64